See https://github.com/rl3279/twitter_sentiment_analysis for organized code. 

Main code used in this notebook is distributed into two files:
- [`preprocessing.py`](https://github.com/rl3279/twitter_sentiment_analysis/blob/main/preprocessing.py), containing all scripts to clean and to process raw tweeter text string. 
- [`feature_engineering.py`](https://github.com/rl3279/twitter_sentiment_analysis/blob/main/feature_engineering.py), containing scripts to extract features from the processed text.

For reference, the exact content of these files will be attached at the very end of this notebook.

In [1]:
import my_globals
from preprocessing import cleaning, preprocess_pipeline
import feature_engineering as fe
import pandas as pd
import numpy as np

This notebook will only showcase the cleaning, preprocessing, feature-engineering workflow on a subset of data, since the resulting dataset is enormous.

In [2]:
from utils import get_sub_dataset, get_sub_featured_datasets
DATA_PATH = "/".join([my_globals.DATA_DIR, my_globals.MAIN_DATA_NAME])
data = get_sub_featured_datasets(size = 5000, random_seed=4)
data

C:\Users\linru\AppData\Roaming\Python\Python39\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,index,target,ids,date,user,text,weekday_Mon,weekday_Tue,weekday_Wed,weekday_Thu,...,tfidf_ð½ð¾ðµ,tfidf_ð½ñ,tfidf_ð¾,tfidf_ð¾ð,tfidf_ð¾ð²,tfidf_ð¾ðµ,tfidf_ð¾ðµð½,tfidf_ð¾ð½ð¾ð¹,tfidf_ð¾ð½ñ,tfidf_ð¾ñ
0,1031776,4,1933110114,Tue May 26 23:01:24 PDT 2009,WookieeChew,@jerryfee Interesting....I'll give it a shot!,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,720217,0,2260825531,Sat Jun 20 19:49:58 PDT 2009,xokristinaxo,Just got back from the Puerto Rican Fest. Mmmm...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1579676,4,2189971972,Tue Jun 16 01:24:27 PDT 2009,Curlybird1988,working today but 2 days off from tomro... lif...,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,183731,0,1967498264,Fri May 29 19:41:29 PDT 2009,JiMpiSh,@tommytrc @tvorse @LisaRedShoesPR The more I h...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1120719,4,1974001323,Sat May 30 12:14:48 PDT 2009,samsonlion,What a beautiful day. Too bad I missed half of...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,112171,0,1825363871,Sun May 17 05:25:27 PDT 2009,xoxolinzieoxox,i cnt believe how much tweets ive done bout th...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,645683,0,2236341913,Fri Jun 19 03:27:52 PDT 2009,Cranialstrain,"@o2 Guys, loving my iPhone but have to say the...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,1011918,4,1881149672,Fri May 22 03:45:55 PDT 2009,reetsjel,&quot;Do You Feel The Way You Hate? Do You Hat...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,1566265,4,2187731611,Mon Jun 15 20:43:13 PDT 2009,ZolaStapler,My blood test was excellent I had my last c...,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
"/".join([my_globals.DATA_DIR, "features.csv"])

data.to_csv()

## Pre-processing 
steps include:
- Link deletion
- Username mentions deletion
- Decontraction ("can't" -> "cannot")
- Lemmetization ("shoes" -> "shoe"; "mice" -> "mouse")
- Stopwords deletion (*e.g.* "a", "the", "to", "of", etc.)
- Punctuation deletion
- Digits deletion





[`preprocessing.py`](https://github.com/rl3279/twitter_sentiment_analysis/blob/main/preprocessing.py) content:

In [3]:
import my_globals
import pandas as pd
import numpy as np
from typing import List
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re
import contractions
from functools import reduce
from dateutil.parser import parse


def setup_nltk():
    """Downloads necessary packages within nltk"""
    packages = ["punkt", "wordnet", "stopwords"]
    for p in packages:
        try:
            nltk.data.find(p)
        except LookupError:
            nltk.download(p)


def tokenize(s: str, how="word_tokenize") -> List[str]:
    if how == "word_tokenize":
        return word_tokenize(s)
    elif how == "split":
        return s.split()


def del_username(s: str) -> str:
    """Delete @Username from a tweet str.

    :param s: input string
    :type s: str
    :rtype: str
    """

    return " ".join([t for t in tokenize(s, how="split") if not t.startswith("@")])


def del_punc(s: str) -> str:
    """Delete punctuations from str.

    :param s: input string
    :type s: str
    :rtype: str
    """
    punc = my_globals.PUNCS
    return "".join([w for w in s if w not in punc])


def del_link(s: str) -> str:
    """Delete links from str.

    :param s: input string
    :type s: str
    :rtype: str
    """
    r = r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)"
    return " ".join([re.sub(r, "", t) for t in tokenize(s, how="split")])


def decontract(s: str) -> str:
    """Remove contractions in text.
    e.g. I'm -> I am; she'd -> she would

    :param s: input string
    :type s: str
    :rtype: str
    """
    tokens = []
    for t in tokenize(s, how="split"):
        tokens.append(contractions.fix(t))
    return " ".join(tokens)


def del_stopwords(s: str) -> str:
    """Delete stopwords and punctuation from a string.
    Note that the type-hinting indicates that this function ought
    to be run first in the pre-processing pipeline.

    :param s: input string
    :type s: str
    """
    stop_words = set(stopwords.words('english'))

    return " ".join([t for t in tokenize(s) if t not in stop_words])


def del_digits(s: str) -> str:
    """Delete digits from str.

    :param s: input string
    :type s: str
    :rtype: str
    """
    return " ".join([w for w in tokenize(s) if not w.isdigit()])


def lemmatize(s: str) -> str:
    """Lemmatize str.

    :param s: input string
    :type s: str
    :rtype: str
    """
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(t) for t in tokenize(s)])


def preprocess_pipeline(s: str, return_lower = True) -> str:
    """Run string through all pre-processing functions.

    :param s: input string
    :type s: str
    :rtype: str
    """
    s = reduce(
        lambda value, function: function(value),
        (
            del_link,
            del_username,
            decontract,
            lemmatize,
            del_stopwords,
            del_punc,
            del_digits
        ),
        s,
    )

    return s.lower() if return_lower else s

def str_datetime(s: str):
    """Parse and format a datetime str to weekday and datetime.MAXYEAR
    
    :param s: input string containing datetime information
    :type s: str
    :rtype: tuple[str, str]
    """
    ss = parse(s).strftime('%a %Y-%m-%d %H:%M:%S')
    return ss[:3], ss[4:]


def cleaning(df: pd.DataFrame):
    """Cleaning script of the data (or subset).
    
    :param df: input dataframe.
    :type df: pd.DataFrame
    :rtype: pd.DataFrame
    """
    # Parse weekday and datetime
    weekday_datetime = pd.DataFrame(
        list(df.loc[:, "date"].apply(str_datetime)),
        columns=["weekday", "datetime"]
    )
    # One-hot encode weekday
    weekdaydummies = pd.get_dummies(
        weekday_datetime['weekday'], 
        prefix='weekday', 
        dtype=float
    )
    weekdaydummies = pd.DataFrame(
        weekdaydummies, 
        columns=['weekday_'+w for w in [
            "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"
        ]]
    )
    # Concatenate weekday dummies to other features
    weekdaydummies_datetime = pd.concat(
        [weekdaydummies, weekday_datetime['datetime']], 
        axis=1
    )
    df = pd.concat([df, weekdaydummies_datetime], axis=1)
    # Drop the column with single unique value.
    df.drop("flag", axis = 1, inplace=True)
    return df


# Feature engineering

proposed features include:
- "!" usage frequency
- "@" usage count 
- capitalized letter usage count
- TF-IDF (Term Frequency - Inverse Document Frequency) from processed text.

[`feature_engineering.py`](https://github.com/rl3279/twitter_sentiment_analysis/blob/main/feature_engineering.py) content:

In [ ]:
import numpy as np
import pandas as pd
from preprocessing import preprocess_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


def exlaim_freq(s: str) -> float:
    """Frequency of excalamtion points in a tweet.

    :param s: input str
    :type s: str
    :rtype: float
    """
    s = "".join(s.split())
    count = sum([1 if t == "!" else 0 for t in s])
    return count / len(s)


def mention_count(s: str) -> int:
    """Counts how many mentions occurred in a tweet.

    :param s: input str
    :type s: str
    :rtype: int
    """
    count = sum([1 if t.startswith("@") else 0 for t in s.split()])
    return count


def cap_freq(s: str) -> float:
    """Frequency of capitalized letter usage in a tweet.

    :param s: input str
    :type s: str
    :rtype: float
    """
    s = preprocess_pipeline(s)
    count = sum([1 if t.isupper() else 0 for t in s])
    return count / len(s)


def get_tfidf(data: pd.Series) -> np.ndarray:
    """Encode a Series of text string to TF-IDF.

    :param data: input data
    :type data: pd.Series
    :rtype: np.ndarray
    """
    vectorizer = CountVectorizer()
    tfidf = TfidfTransformer()
    X = vectorizer.fit_transform(data)
    X = tfidf.fit_transform(X)
    return X.toarray()
